In [1]:
!pip show transformers accelerate

Name: transformers
Version: 4.38.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/conda/lib/python3.11/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: autogluon.multimodal, autogluon.timeseries
---
Name: accelerate
Version: 0.21.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /opt/conda/lib/python3.11/site-packages
Requires: numpy, packaging, psutil, pyyaml, torch
Required-by: autogluon.multimodal, autogluon.timeseries


In [2]:
%pip install --upgrade transformers accelerate

  Using cached transformers-4.46.2-py3-none-any.whl.metadata (44 kB)
  Using cached accelerate-1.1.1-py3-none-any.whl.metadata (19 kB)
  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.46.2-py3-none-any.whl (10.0 MB)
Using cached accelerate-1.1.1-py3-none-any.whl (333 kB)
Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
ERROR: pip's depend

In [2]:
import os
import pandas as pd
import datasets
from datasets import Dataset
from copy import deepcopy
import time
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from datasets import load_dataset
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from accelerate import Accelerator
import nltk
from nltk.corpus import wordnet
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    AutoTokenizer, 
    AutoImageProcessor,  
    AutoModel,
    TrainingArguments, 
    Trainer,
    logging
)

2024-11-17 06:17:22.830783: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-17 06:17:22.845426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-17 06:17:22.863900: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-17 06:17:22.869595: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-17 06:17:22.883158: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# Read the answer space
with open(os.path.join("dataset", "answer_space.txt")) as f:
    answer_space = f.read().splitlines()

# Read the files and process the answer column
train_df = pd.read_csv(os.path.join("dataset", "data_train.csv"))
test_df = pd.read_csv(os.path.join("dataset", "data_eval.csv"))

# Function to get first answer and its index
def process_answer(answer):
    first_answer = answer.replace(" ", "").split(",")[0]
    return answer_space.index(first_answer)

# Apply the processing to both dataframes
train_df['label'] = train_df['answer'].apply(process_answer)
test_df['label'] = test_df['answer'].apply(process_answer)

# Convert to datasets format
original_dataset = datasets.DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'test': Dataset.from_pandas(test_df)
})

In [4]:
dataset = original_dataset

In [47]:
n_rows_train = 8*100
n_rows_test = 2*100

# Select the first n rows from each set
train_sample = original_dataset['train'].select(range(n_rows_train))
test_sample = original_dataset['test'].select(range(n_rows_test))

# Create a smaller DatasetDict with the samples
small_dataset = {
    "train": train_sample,
    "test": test_sample
}

dataset = small_dataset

In [5]:
nltk.download('wordnet')
# Disable MLflow logging
os.environ['DISABLE_MLFLOW_INTEGRATION'] = 'TRUE'


class MultimodalVQAModel(nn.Module):
    def __init__(
            self,
            num_labels: int = len(answer_space),
            intermediate_dim: int = 512,
            pretrained_text_name: str = 'bert-base-uncased',
            pretrained_image_name: str = 'google/vit-base-patch16-224-in21k'):

        super(MultimodalVQAModel, self).__init__()
        self.num_labels = num_labels
        self.pretrained_text_name = pretrained_text_name
        self.pretrained_image_name = pretrained_image_name

        self.text_encoder = AutoModel.from_pretrained(
            self.pretrained_text_name,
            force_download=True
        )
        self.image_encoder = AutoModel.from_pretrained(
            self.pretrained_image_name,
            force_download=True
        )
        self.fusion = nn.Sequential(
            nn.Linear(self.text_encoder.config.hidden_size + self.image_encoder.config.hidden_size, intermediate_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
        )

        self.classifier = nn.Linear(intermediate_dim, self.num_labels)
        self.criterion = nn.CrossEntropyLoss()

    def forward(
            self,
            input_ids: torch.LongTensor,
            pixel_values: torch.FloatTensor,
            attention_mask: Optional[torch.LongTensor] = None,
            token_type_ids: Optional[torch.LongTensor] = None,
            labels: Optional[torch.LongTensor] = None):

        encoded_text = self.text_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True,
        )
        encoded_image = self.image_encoder(
            pixel_values=pixel_values,
            return_dict=True,
        )
        fused_output = self.fusion(
            torch.cat(
                [
                    encoded_text['pooler_output'],
                    encoded_image['pooler_output'],
                ],
                dim=1
            )
        )
        logits = self.classifier(fused_output)

        out = {
            "logits": logits
        }
        if labels is not None:
            loss = self.criterion(logits, labels)
            out["loss"] = loss

        return out

@dataclass
class MultimodalCollator:
    tokenizer: AutoTokenizer
    preprocessor: AutoImageProcessor

    def tokenize_text(self, texts: List[str]):
        encoded_text = self.tokenizer(
            text=texts,
            padding='longest',
            max_length=24,
            truncation=True,
            return_tensors='pt',
            return_token_type_ids=True,
            return_attention_mask=True,
        )
        return {
            "input_ids": encoded_text['input_ids'].squeeze(),
            "token_type_ids": encoded_text['token_type_ids'].squeeze(),
            "attention_mask": encoded_text['attention_mask'].squeeze(),
        }

    def preprocess_images(self, images: List[str]):
        processed_images = self.preprocessor(
            images=[Image.open(os.path.join("dataset", "images", image_id + ".png")).convert('RGB') for image_id in images],
            return_tensors="pt",
        )
        return {
            "pixel_values": processed_images['pixel_values'].squeeze(),
        }

    def __call__(self, raw_batch_dict):
        return {
            **self.tokenize_text(
                raw_batch_dict['question']
                if isinstance(raw_batch_dict, dict) else
                [i['question'] for i in raw_batch_dict]
            ),
            **self.preprocess_images(
                raw_batch_dict['image_id']
                if isinstance(raw_batch_dict, dict) else
                [i['image_id'] for i in raw_batch_dict]
            ),
            'labels': torch.tensor(
                raw_batch_dict['label']
                if isinstance(raw_batch_dict, dict) else
                [i['label'] for i in raw_batch_dict],
                dtype=torch.int64
            ),
        }

def wup_measure(a,b,similarity_threshold=0.925):
    """
    Returns Wu-Palmer similarity score.
    More specifically, it computes:
        max_{x \in interp(a)} max_{y \in interp(b)} wup(x,y)
        where interp is a 'interpretation field'
    """
    def get_semantic_field(a):
        weight = 1.0
        semantic_field = wordnet.synsets(a,pos=wordnet.NOUN)
        return (semantic_field,weight)


    def get_stem_word(a):
        """
        Sometimes answer has form word\d+:wordid.
        If so we return word and downweight
        """
        weight = 1.0
        return (a,weight)


    global_weight=1.0

    (a,global_weight_a)=get_stem_word(a)
    (b,global_weight_b)=get_stem_word(b)
    global_weight = min(global_weight_a,global_weight_b)

    if a==b:
        # they are the same
        return 1.0*global_weight

    if a==[] or b==[]:
        return 0


    interp_a,weight_a = get_semantic_field(a)
    interp_b,weight_b = get_semantic_field(b)

    if interp_a == [] or interp_b == []:
        return 0

    # we take the most optimistic interpretation
    global_max=0.0
    for x in interp_a:
        for y in interp_b:
            local_score=x.wup_similarity(y)
            if local_score > global_max:
                global_max=local_score

    # we need to use the semantic fields and therefore we downweight
    # unless the score is high which indicates both are synonyms
    if global_max < similarity_threshold:
        interp_weight = 0.1
    else:
        interp_weight = 1.0

    final_score=global_max*weight_a*weight_b*interp_weight*global_weight
    return final_score

def batch_wup_measure(labels, preds):
    wup_scores = [wup_measure(answer_space[label], answer_space[pred]) for label, pred in zip(labels, preds)]
    return np.mean(wup_scores)

def compute_metrics(eval_tuple: Tuple[np.ndarray, np.ndarray]) -> Dict[str, float]:
    logits, labels = eval_tuple
    preds = logits.argmax(axis=-1)
    return {
        "wups": batch_wup_measure(labels, preds),
        "acc": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average='macro')
    }

def createMultimodalVQACollatorAndModel(text='bert-base-uncased', image='google/vit-base-patch16-224-in21k'):
    tokenizer = AutoTokenizer.from_pretrained(text, force_download=True)
    preprocessor = AutoImageProcessor.from_pretrained(image, force_download=True, use_fast=True )

    multi_collator = MultimodalCollator(
        tokenizer=tokenizer,
        preprocessor=preprocessor,
    )

    multi_model = MultimodalVQAModel(pretrained_text_name=text, pretrained_image_name=image).to(device)
    return multi_collator, multi_model

def createAndTrainModel(dataset, args, text_model='bert-base-uncased',
                       image_model='google/vit-base-patch16-224-in21k',
                       multimodal_model='bert_vit'):
    # Create output directory with timestamp to ensure uniqueness
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    output_dir = os.path.join("checkpoint", multimodal_model, f"run_{timestamp}")
    os.makedirs(output_dir, exist_ok=True)

    collator, model = createMultimodalVQACollatorAndModel(text_model, image_model)

    # Create a new TrainingArguments object with unique run name
    training_args = TrainingArguments(
        output_dir=output_dir,
        run_name=f"run_{timestamp}",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="wups",
        greater_is_better=True,
        per_device_train_batch_size=args.per_device_train_batch_size if hasattr(args, 'per_device_train_batch_size') else 8,
        per_device_eval_batch_size=args.per_device_eval_batch_size if hasattr(args, 'per_device_eval_batch_size') else 8,
        num_train_epochs=args.num_train_epochs if hasattr(args, 'num_train_epochs') else 3,
        learning_rate=args.learning_rate if hasattr(args, 'learning_rate') else 5e-5,
        report_to=[],
        remove_unused_columns=False,  # Important: Keep all columns
    )

    # Verify dataset format
    print("Sample training example:", dataset['train'][0])
    print("Sample test example:", dataset['test'][0])

    multi_trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        data_collator=collator,
        compute_metrics=compute_metrics,
    )

    train_multi_metrics = multi_trainer.train()
    eval_multi_metrics = multi_trainer.evaluate()

    # Save the final model explicitly
    model_save_path = os.path.join(output_dir, "final_model.pt")
    torch.save({
        'model_state_dict': model.state_dict(),
        'train_metrics': train_multi_metrics,
        'eval_metrics': eval_multi_metrics
    }, model_save_path)

    return collator, model, train_multi_metrics, eval_multi_metrics

# Set device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Device")
print(device)

# Training arguments
args = TrainingArguments(
    output_dir="checkpoint",
    seed=42,
    eval_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=3,
    metric_for_best_model='wups',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    remove_unused_columns=False,
    num_train_epochs=5,
    fp16=True,
    dataloader_num_workers=8,
    load_best_model_at_end=True,
)

[nltk_data] Downloading package wordnet to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Device
cuda:0


In [ ]:
collator, model, train_multi_metrics, eval_multi_metrics = createAndTrainModel(dataset, args)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Sample training example: {'question': 'what is the object on the shelves', 'answer': 'cup', 'image_id': 'image100', 'label': 149}
Sample test example: {'question': 'what is the colour of the bag on the chair', 'answer': 'pink', 'image_id': 'image399', 'label': 387}


Epoch,Training Loss,Validation Loss


In [5]:
labels = np.random.randint(len(answer_space), size=5)
preds = np.random.randint(len(answer_space), size=5)

def showAnswers(ids):
    print([answer_space[id] for id in ids])

showAnswers(labels)
showAnswers(preds)

print("Predictions vs Labels: ", batch_wup_measure(labels, preds))
print("Labels vs Labels: ", batch_wup_measure(labels, labels))

['ceiling', 'shelves', 'head_phones', 'chair', 'box_of_paper']
['air_conditioner', 'clothing_hamper', 'stamp', 'fire_extinguisher', 'medal']
Predictions vs Labels:  0.02608187134502924
Labels vs Labels:  1.0


In [6]:
eval_multi_metrics

{'eval_loss': 3.4556491374969482,
 'eval_wups': 0.3071833106722724,
 'eval_acc': 0.2610264635124298,
 'eval_f1': 0.03549650549105638,
 'eval_runtime': 39.9818,
 'eval_samples_per_second': 62.378,
 'eval_steps_per_second': 0.975,
 'epoch': 10.0}

# Load and test model

In [ ]:
model = MultimodalVQAModel()
# RTC:checkpoint/bert_vit/run_20241116-153943/final_model.pt
checkpoint_path = os.path.join("checkpoint", "bert_vit", "final", "full_64_64_10.pt")
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    print("Model loaded successfully")
    print("Final evaluation metrics:", checkpoint['eval_metrics'])
else:
    print("No saved model found at:", checkpoint_path)

In [ ]:
sample = collator(dataset["test"][100:1])

input_ids = sample["input_ids"].to(device)
token_type_ids = sample["token_type_ids"].to(device)
attention_mask = sample["attention_mask"].to(device)
pixel_values = sample["pixel_values"].to(device)
labels = sample["labels"].to(device)

In [ ]:
model.eval()
output = model(input_ids, pixel_values, attention_mask, token_type_ids, labels)

In [ ]:
preds = output["logits"].argmax(axis=-1).cpu().numpy()
preds

In [16]:
from IPython.display import display

def showExample(train=True, id=None):
    if train:
        data = dataset["train"]
    else:
        data = dataset["test"]
    if id == None:
        id = np.random.randint(len(data))
    image = Image.open(os.path.join("dataset", "images", data[id]["image_id"] + ".png"))
    display(image)

    print("Question:\t", data[id]["question"])
    print("Answer:\t\t", data[id]["answer"], "(Label: {0})".format(data[id]["label"]))

In [ ]:
for i in range(000, 005):
    print("*********************************************************")
    showExample(train=False, id=i)
    print("Predicted Answer:\t", answer_space[preds[i-1000]])
    print("*********************************************************")